In [1]:
import torch
import torch.nn as nn

batch_size, hidden_size =4, 512

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.z1 = None
        self.w1 = nn.Linear(hidden_size, hidden_size, bias=False)
        self.w2 = nn.Linear(hidden_size, 1, bias=False)
    
    def forward(self, x):
        z1 = self.w1(x)
        print(z1)
        self.z1 = z1.clone().detach()
        z2 = self.w2(z1)
        return z2

model = Net()

In [2]:
import torch
import torch.nn as nn

batch_size, hidden_size =4, 512

class Net1(nn.Module):
    def __init__(self):
        super().__init__()
        self.z1 = None
        self.w1 = nn.Linear(512, 100, bias=False)
        self.w2 = nn.Linear(100, 1024, bias=False)
        self.w3 = nn.Linear(1024, 10, bias=False)
    
    def forward(self, x):
        z1 = self.w1(x)
        print(z1)
        self.z1 = z1.clone().detach()
        z2 = self.w2(z1)
        z3 = self.w3(z2)
        return z3

model1 = Net1()
print(model1.w1.weight.shape)
print(model1.w2.weight.shape)
print(model1.w3.weight.shape)

x = torch.randn(4, 512)
print(x.shape)
out = model1(x)
print(out.shape)

torch.Size([100, 512])
torch.Size([1024, 100])
torch.Size([10, 1024])
torch.Size([4, 512])
tensor([[ 1.2053e-02, -2.8138e-01,  1.1684e+00,  2.8869e-01, -2.3495e-01,
          1.7711e-02, -9.4961e-01, -4.5591e-01,  4.5444e-01, -1.0913e+00,
          2.5712e-01,  6.3148e-01, -3.0727e-01, -2.2693e-01, -7.2851e-01,
          4.4833e-01, -7.1848e-01, -5.5857e-01,  2.7255e-01,  4.4720e-01,
          4.0542e-01,  8.0266e-01, -2.6425e-01,  9.6507e-01, -8.0444e-01,
          9.4456e-01,  1.8210e-01,  2.4276e-01, -4.8592e-01,  1.8035e-01,
          7.0945e-01,  2.7731e-01,  6.1586e-01,  7.2934e-01,  2.7688e-01,
         -1.2357e-01, -2.0440e-01,  2.0607e+00,  1.8939e-01,  1.2655e+00,
         -5.0912e-01,  2.0741e-01,  5.9439e-01,  5.5814e-01,  7.1868e-04,
         -1.7160e-01, -3.5857e-01,  7.2398e-01,  7.5328e-01, -1.1581e-02,
          3.5739e-01,  4.9360e-01, -9.1885e-01, -9.0178e-01,  7.6294e-01,
         -1.2743e+00, -3.4529e-01, -2.0674e-01,  1.3492e-01, -4.7945e-01,
          1.2097e+00,

In [3]:
torch.matmul(x, model1.w1.weight.T).shape

torch.Size([4, 100])

In [4]:
(x@model1.w1.weight.T).shape

torch.Size([4, 100])

In [5]:

import torch

# example input sizes
batch_size, hidden_size = 4, 512

# create dummy data (bsz=4, hid=256)
x = torch.randn(batch_size,hidden_size) 
out = model(x)
print(out)

tensor([[-0.3885, -0.3268,  0.7740,  ...,  0.3769, -0.9833, -0.0972],
        [ 1.8091,  0.0673,  0.8450,  ..., -0.3477, -0.6267, -0.7226],
        [ 0.4881,  0.0083,  0.8602,  ..., -0.6376,  0.0209, -0.4623],
        [-0.4124, -0.4325,  1.5413,  ...,  0.0395, -0.1859, -0.2388]],
       grad_fn=<MmBackward0>)
tensor([[ 0.1055],
        [ 0.1254],
        [-0.5648],
        [ 0.0886]], grad_fn=<MmBackward0>)


In [6]:
from torch.optim import SGD

optimizer = SGD(model.parameters(), lr=1e-1)

In [7]:
# craete dummy data (bsz=4)
y = torch.tensor([[1.9], [9.5], [0.9], [1.2]])

# compute mean square error loss
L = torch.nn.functional.mse_loss(out, y)

# check dtype of loss
f"loss type = {L.dtype}"

'loss type = torch.float32'

In [8]:
print(L)

tensor(23.6211, grad_fn=<MseLossBackward0>)


In [9]:
loss = torch.sum((out-y)**2)/batch_size
print(loss)

tensor(23.6211, grad_fn=<DivBackward0>)


In [10]:
DL_Dout = 2*(out - y) / batch_size # Dout/DL
print(DL_Dout) # Backward Activation [1,4] [output_size, batch_size]

tensor([[-0.8972],
        [-4.6873],
        [-0.7324],
        [-0.5557]], grad_fn=<DivBackward0>)


In [11]:

Dout_Dw2 = model.z1.clone().detach() # Backward Activation: w2 layer
Dout_Dw2.shape

torch.Size([4, 512])

In [12]:
DL_Dw2 = torch.matmul(DL_Dout.T, Dout_Dw2) # w2 gradient 
DL_Dw2.shape

torch.Size([1, 512])

In [13]:
DL_Dz1 = torch.matmul(DL_Dout, model.w2.weight) # Dl/Dout * w2
DL_Dz1.shape

torch.Size([4, 512])

In [14]:
# x.shape [batch_size=4, hidden_size=512]
DL_Dw1 = torch.matmul(DL_Dz1.T,x) # w1 gradient
DL_Dw1.shape

torch.Size([512, 512])

In [15]:
L.backward()

In [16]:
print(model.w1.weight.grad)
print(DL_Dw1)

tensor([[-0.0856,  0.0118, -0.0254,  ..., -0.0425,  0.0192, -0.0561],
        [ 0.1334, -0.0184,  0.0396,  ...,  0.0661, -0.0298,  0.0874],
        [ 0.0753, -0.0104,  0.0224,  ...,  0.0373, -0.0169,  0.0493],
        ...,
        [ 0.2326, -0.0321,  0.0691,  ...,  0.1153, -0.0521,  0.1524],
        [ 0.2156, -0.0297,  0.0640,  ...,  0.1069, -0.0483,  0.1413],
        [ 0.0517, -0.0071,  0.0154,  ...,  0.0257, -0.0116,  0.0339]])
tensor([[-0.0856,  0.0118, -0.0254,  ..., -0.0425,  0.0192, -0.0561],
        [ 0.1334, -0.0184,  0.0396,  ...,  0.0661, -0.0298,  0.0874],
        [ 0.0753, -0.0104,  0.0224,  ...,  0.0373, -0.0169,  0.0493],
        ...,
        [ 0.2326, -0.0321,  0.0691,  ...,  0.1153, -0.0521,  0.1524],
        [ 0.2156, -0.0297,  0.0640,  ...,  0.1069, -0.0483,  0.1413],
        [ 0.0517, -0.0071,  0.0154,  ...,  0.0257, -0.0116,  0.0339]],
       grad_fn=<MmBackward0>)


In [17]:
print(f'before: {model.w1.weight}\n')
optimizer.step()
#print(model.w1.weight.grad)
print(f'after: {model.w1.weight}\n')

before: Parameter containing:
tensor([[-0.0021,  0.0054, -0.0359,  ...,  0.0298,  0.0162, -0.0093],
        [-0.0225,  0.0289,  0.0428,  ...,  0.0317,  0.0185,  0.0310],
        [ 0.0394,  0.0035,  0.0282,  ..., -0.0349, -0.0162, -0.0333],
        ...,
        [ 0.0064, -0.0342,  0.0275,  ..., -0.0055, -0.0226, -0.0247],
        [-0.0439,  0.0092, -0.0210,  ...,  0.0023, -0.0396,  0.0100],
        [-0.0334,  0.0217, -0.0039,  ..., -0.0278,  0.0370,  0.0168]],
       requires_grad=True)

after: Parameter containing:
tensor([[ 0.0064,  0.0042, -0.0333,  ...,  0.0341,  0.0142, -0.0037],
        [-0.0358,  0.0307,  0.0388,  ...,  0.0251,  0.0214,  0.0223],
        [ 0.0319,  0.0045,  0.0260,  ..., -0.0386, -0.0145, -0.0382],
        ...,
        [-0.0169, -0.0310,  0.0206,  ..., -0.0170, -0.0174, -0.0400],
        [-0.0654,  0.0122, -0.0274,  ..., -0.0084, -0.0347, -0.0041],
        [-0.0386,  0.0224, -0.0055,  ..., -0.0303,  0.0381,  0.0135]],
       requires_grad=True)



In [18]:
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [19]:
optimizer.param_groups[0]['params'][0] # w1 weight

Parameter containing:
tensor([[ 0.0064,  0.0042, -0.0333,  ...,  0.0341,  0.0142, -0.0037],
        [-0.0358,  0.0307,  0.0388,  ...,  0.0251,  0.0214,  0.0223],
        [ 0.0319,  0.0045,  0.0260,  ..., -0.0386, -0.0145, -0.0382],
        ...,
        [-0.0169, -0.0310,  0.0206,  ..., -0.0170, -0.0174, -0.0400],
        [-0.0654,  0.0122, -0.0274,  ..., -0.0084, -0.0347, -0.0041],
        [-0.0386,  0.0224, -0.0055,  ..., -0.0303,  0.0381,  0.0135]],
       requires_grad=True)

In [20]:
#optimizer.param_groups[0]['params'][1] #w2 weight

In [21]:
import torch
import torch.nn as nn

batch_size, hidden_size =4, 512

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.z1 = None
        self.w1 = nn.Linear(hidden_size, hidden_size, bias=False)
        self.w2 = nn.Linear(hidden_size, 1, bias=False)
    
    def forward(self, x):
        z1 = self.w1(x)
        print(z1)
        self.z1 = z1.clone().detach()
        z2 = self.w2(z1)
        return z2

model = Net()

# example input sizes
batch_size, hidden_size = 4, 512

# create dummy data (bsz=4, hid=256)
x = torch.randn(batch_size,hidden_size) 
out = model(x)
#print(out)

from torch.optim import RMSprop

optimizer = RMSprop(model.parameters(), lr=1e-1)

# craete dummy data (bsz=4)
y = torch.tensor([[1.9], [9.5], [0.9], [1.2]])

# compute mean square error loss
L = torch.nn.functional.mse_loss(out, y)

# check dtype of loss
#f"loss type = {L.dtype}"
L.backward()

print(f'before: {model.w1.weight}\n')
optimizer.step()
#print(model.w1.weight.grad)
print(f'after: {model.w1.weight}\n')

tensor([[ 0.6451, -0.5995,  0.1066,  ..., -0.7793,  1.1848,  0.0143],
        [ 0.3115,  1.2819,  1.0466,  ..., -0.2717,  0.3988,  0.1265],
        [-0.4989,  0.0384, -0.2930,  ..., -0.7682,  0.4926,  0.3768],
        [ 0.4781,  0.1196, -0.4912,  ...,  0.0387, -0.3887, -0.2607]],
       grad_fn=<MmBackward0>)
before: Parameter containing:
tensor([[-0.0428,  0.0275, -0.0161,  ..., -0.0036,  0.0251,  0.0356],
        [-0.0376,  0.0034,  0.0349,  ..., -0.0024,  0.0207,  0.0341],
        [ 0.0212,  0.0041, -0.0154,  ...,  0.0037,  0.0003,  0.0075],
        ...,
        [-0.0059,  0.0205,  0.0327,  ...,  0.0228, -0.0251, -0.0077],
        [ 0.0258, -0.0422,  0.0310,  ..., -0.0296, -0.0283, -0.0155],
        [ 0.0249, -0.0296,  0.0196,  ..., -0.0062,  0.0433, -0.0263]],
       requires_grad=True)

after: Parameter containing:
tensor([[-1.0428, -0.9725, -1.0161,  ..., -1.0036,  1.0251, -0.9644],
        [ 0.9624,  1.0034,  1.0349,  ...,  0.9976, -0.9793,  1.0341],
        [ 1.0212,  1.0041,  

In [22]:
#optimizer.param_groups

In [23]:
from torch.optim import SGD

fp32_model= Net().to("cuda")
lr = 1e-0
optimizer = SGD(fp32_model.parameters(), lr=lr)
# print(lr)  #1.0

In [24]:
f"GPU = {torch.cuda.memory_allocated(0) / (1024 ** 2)} MiB"

'GPU = 1.001953125 MiB'

In [25]:
fp32_model.w1.weight

Parameter containing:
tensor([[-0.0090,  0.0216, -0.0142,  ...,  0.0296,  0.0021, -0.0142],
        [-0.0063, -0.0333,  0.0339,  ..., -0.0066, -0.0135, -0.0414],
        [ 0.0342,  0.0385, -0.0415,  ..., -0.0416, -0.0054, -0.0287],
        ...,
        [ 0.0410,  0.0120,  0.0190,  ...,  0.0098, -0.0418,  0.0419],
        [-0.0006,  0.0331,  0.0249,  ...,  0.0122,  0.0266, -0.0083],
        [-0.0066, -0.0123, -0.0396,  ..., -0.0416, -0.0441,  0.0115]],
       device='cuda:0', requires_grad=True)

In [26]:
#fp32_model.w2.weight

In [27]:
import torch

# example input sizes
#batch_size, hidden_size =4, 8

# create dummy data (bsz=4, hid=256)
x = torch.randn(batch_size,hidden_size, dtype=torch.float, device="cuda") 

# do forward
z2 = fp32_model(x)

# check dtypr of output logits
f"logits type = {z2.dtype}"

tensor([[ 0.0027,  0.1247,  0.6845,  ...,  0.2222,  0.3790,  0.0798],
        [-0.5664, -0.8630, -0.5983,  ..., -0.0732,  0.0292, -1.0817],
        [ 0.1397,  0.5587, -0.2522,  ..., -1.0445,  0.6003,  0.7344],
        [ 0.4070, -0.2195,  0.7548,  ...,  0.2692,  0.7560,  0.6452]],
       device='cuda:0', grad_fn=<MmBackward0>)


'logits type = torch.float32'

In [28]:
# craete dummy data (bsz=4)
#y = torch.tensor([[1.9], [9.5], [0.9], [1.2]], dtype=torch.half, device="cuda") #batch_size =4
y = torch.tensor([[1.9], [9.5], [0.9], [1.2]], dtype=torch.float32, device="cuda") #batch_size =4
#y = torch.tensor([[1.9]], dtype=torch.float32, device="cuda")
#y = torch.tensor([[1.9], [0.5]], dtype=torch.float32, device="cuda")
# compute mean square error loss
L = torch.nn.functional.mse_loss(z2, y)

# check dtype of loss
f"loss type = {L.dtype}"

'loss type = torch.float32'

In [29]:
print(L)
print(z2)
print(y)
loss = torch.sum((z2-y)**2/batch_size)
print(loss)

tensor(23.4768, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([[ 0.2409],
        [ 0.1095],
        [-0.2919],
        [-0.0456]], device='cuda:0', grad_fn=<MmBackward0>)
tensor([[1.9000],
        [9.5000],
        [0.9000],
        [1.2000]], device='cuda:0')
tensor(23.4768, device='cuda:0', grad_fn=<SumBackward0>)


In [30]:
L.backward()
w2_weight = fp32_model.w2.weight.clone().detach()
w1_weight = fp32_model.w1.weight.clone().detach()
print(f'before: {fp32_model.w2.weight}\n')
optimizer.step()
print(f'after: {fp32_model.w2.weight}\n')

before: Parameter containing:
tensor([[ 1.7143e-02, -4.8008e-03,  1.2542e-02,  2.2025e-02,  2.8882e-02,
          1.8568e-02, -2.8732e-02,  1.5700e-02,  9.7479e-03, -2.9827e-02,
          2.0754e-02, -3.6667e-02, -1.5246e-02, -1.4673e-02, -3.4143e-02,
          1.8870e-02, -3.3328e-02,  1.1338e-02,  2.4312e-03, -4.0811e-03,
         -1.4395e-02, -4.1016e-02, -4.3679e-02, -4.2365e-02, -2.3039e-02,
          1.7028e-02,  9.3678e-03, -2.9893e-02, -2.7834e-02, -3.3187e-02,
         -4.3348e-02,  2.3753e-03, -2.2288e-03,  3.7972e-02,  1.2616e-02,
          3.8895e-02,  2.7593e-02,  1.4104e-02, -3.0829e-02,  1.9050e-02,
          5.9025e-03,  7.7837e-03,  2.6884e-02,  4.1996e-02, -1.4450e-02,
         -2.2397e-02, -5.9393e-03, -2.3913e-05,  3.6137e-02,  3.0431e-02,
          4.0670e-02,  3.6752e-02, -3.9020e-02,  1.5637e-02, -2.7268e-02,
         -1.2595e-02,  2.1426e-02,  9.0879e-05,  2.4869e-02, -4.1285e-02,
         -3.9356e-02,  6.0615e-03,  1.8779e-03,  1.7123e-02,  1.0488e-02,
        

In [31]:
DL_Dz2= 2 * (z2 - y) / batch_size # DL/Dz2  (BWD-activation: layer2), in case of MSE
print(DL_Dz2) # [4,1] [batch_size, output_size=1] DL/Dz2
Dz2_Dw2 = fp32_model.z1.clone().detach() #Dz2/Dw2
print(Dz2_Dw2.shape) #[4,8] [batch_size, hidden_size] 
print(Dz2_Dw2)
#DL_Dw2 = DL_Dz2.T * Dz2_Dw2
DL_Dw2 = torch.matmul(DL_Dz2.T, Dz2_Dw2) #[1,4] * [4,8] batch_size, hidden_size
print(DL_Dw2.shape) # [1,8] [output_size=1, hidden_size]
print(DL_Dw2)
print(w2_weight) # [hidden_size=8 , output_size=1]
print(w2_weight - lr * DL_Dw2)

tensor([[-0.8296],
        [-4.6953],
        [-0.5960],
        [-0.6228]], device='cuda:0', grad_fn=<DivBackward0>)
torch.Size([4, 512])
tensor([[ 0.0027,  0.1247,  0.6845,  ...,  0.2222,  0.3790,  0.0798],
        [-0.5664, -0.8630, -0.5983,  ..., -0.0732,  0.0292, -1.0817],
        [ 0.1397,  0.5587, -0.2522,  ..., -1.0445,  0.6003,  0.7344],
        [ 0.4070, -0.2195,  0.7548,  ...,  0.2692,  0.7560,  0.6452]],
       device='cuda:0')
torch.Size([1, 512])
tensor([[ 2.3202,  3.7521,  1.9218,  1.1778, -1.4855,  2.1561, -3.1861,  6.6767,
          2.1723, -1.2212, -0.2601,  4.2831,  0.2498,  0.5608, -2.2401,  3.8822,
          0.8645,  2.1086,  1.0121, -0.7782, -1.3883,  2.5099, -0.1526, -1.3449,
         -3.4523,  3.2604, -1.1456, -3.0491, -1.7937,  5.7019,  0.7258, -1.4789,
         -0.0271,  1.1386, -3.9155,  3.3305,  6.8583,  7.3380,  2.0006, -0.5187,
         -2.1127,  0.7362,  1.4117, -1.4453,  1.3606, -0.7661, -6.3832,  2.4708,
         -2.8567,  0.9846,  0.4356, -3.9114,  3.6

In [32]:
fp32_model.w1.weight # w1 = [hidden_size, hidden_size] [8,8]

Parameter containing:
tensor([[-0.1225, -0.0400, -0.0139,  ...,  0.0628, -0.0690,  0.0354],
        [ 0.0255, -0.0160,  0.0338,  ..., -0.0159,  0.0065, -0.0553],
        [-0.0489, -0.0066, -0.0412,  ..., -0.0173, -0.0575,  0.0076],
        ...,
        [ 0.2714,  0.1371,  0.0183,  ..., -0.0575,  0.1026, -0.0588],
        [ 0.0063,  0.0368,  0.0248,  ...,  0.0102,  0.0309, -0.0113],
        [ 0.1724,  0.0849, -0.0402,  ..., -0.0939,  0.0681, -0.0668]],
       device='cuda:0', requires_grad=True)

In [33]:
DL_Dz2= 2 * (z2 - y) / batch_size # DL/Dz2  (BWD-activation: layer2), in case of MSE
print(DL_Dz2) # [4,1] [batch_size=4, output_size=1]
print(w2_weight.shape) # [1,8]
temp = torch.matmul(DL_Dz2, w2_weight) #DL/Dz2 * w2
print(temp.shape) # [4,8]
print(x.shape) # [4,8]
DL_Dw1 = torch.matmul(temp.T, x) # [8,4] * [4,8] = [8,8]
print(DL_Dw1.shape) #[8,8]
print(w1_weight - lr * DL_Dw1)

tensor([[-0.8296],
        [-4.6953],
        [-0.5960],
        [-0.6228]], device='cuda:0', grad_fn=<DivBackward0>)
torch.Size([1, 512])
torch.Size([4, 512])
torch.Size([4, 512])
torch.Size([512, 512])
tensor([[-0.1225, -0.0400, -0.0139,  ...,  0.0628, -0.0690,  0.0354],
        [ 0.0255, -0.0160,  0.0338,  ..., -0.0159,  0.0065, -0.0553],
        [-0.0489, -0.0066, -0.0412,  ..., -0.0173, -0.0575,  0.0076],
        ...,
        [ 0.2714,  0.1371,  0.0183,  ..., -0.0575,  0.1026, -0.0588],
        [ 0.0063,  0.0368,  0.0248,  ...,  0.0102,  0.0309, -0.0113],
        [ 0.1724,  0.0849, -0.0402,  ..., -0.0939,  0.0681, -0.0668]],
       device='cuda:0', grad_fn=<SubBackward0>)


In [34]:
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.w1 = nn.Linear(512, 512, bias=False)
        self.w2 = nn.Linear(512, 1, bias=False)
    
    def forward(self, x):
        z1 = self.w1(x)
        z2 = self.w2(z1)
        return z2

from torch.optim import SGD

fp32_model= Net().to("cuda")
optimizer = SGD(fp32_model.parameters(), lr=1e-2)


### Float2Half

In [35]:
fp16_model = Net().half().to("cuda")
fp16_model.load_state_dict(fp32_model.state_dict())

<All keys matched successfully>

### Forward

In [36]:
import torch

# example input sizes
batch_size, hidden_size = 4, 512

# create dummy data (bsz=4, hid=256)
x = torch.randn(batch_size,hidden_size, dtype=torch.half, device="cuda") 

# do forward
z2 = fp16_model(x)

# check dtypr of output logits
f"logits type = {z2.dtype}"

'logits type = torch.float16'

In [37]:
# craete dummy data (bsz=4)
y = torch.tensor([[1.9], [9.5], [0.9], [1.2]], dtype=torch.half, device="cuda")

# compute mean square error loss
L = torch.nn.functional.mse_loss(z2, y)

# check dtype of loss
f"loss type = {L.dtype}"

'loss type = torch.float16'

### Backward

In [38]:
# loss scaling
L *= 1024

# do backward
L.backward()

### Update Weight

In [39]:
print(f'before: {fp32_model.w1.weight}\n')
optimizer.step()
print(f'after: {fp32_model.w1.weight}\n')

before: Parameter containing:
tensor([[ 0.0108,  0.0091,  0.0023,  ..., -0.0054,  0.0047, -0.0132],
        [-0.0140, -0.0182,  0.0112,  ...,  0.0318,  0.0181, -0.0185],
        [-0.0333,  0.0409,  0.0015,  ..., -0.0210, -0.0003, -0.0340],
        ...,
        [ 0.0270,  0.0102,  0.0430,  ..., -0.0134, -0.0132, -0.0054],
        [-0.0426, -0.0003, -0.0204,  ...,  0.0396, -0.0296,  0.0336],
        [ 0.0124, -0.0283,  0.0418,  ..., -0.0426, -0.0158,  0.0190]],
       device='cuda:0', requires_grad=True)

after: Parameter containing:
tensor([[ 0.0108,  0.0091,  0.0023,  ..., -0.0054,  0.0047, -0.0132],
        [-0.0140, -0.0182,  0.0112,  ...,  0.0318,  0.0181, -0.0185],
        [-0.0333,  0.0409,  0.0015,  ..., -0.0210, -0.0003, -0.0340],
        ...,
        [ 0.0270,  0.0102,  0.0430,  ..., -0.0134, -0.0132, -0.0054],
        [-0.0426, -0.0003, -0.0204,  ...,  0.0396, -0.0296,  0.0336],
        [ 0.0124, -0.0283,  0.0418,  ..., -0.0426, -0.0158,  0.0190]],
       device='cuda:0', requ

In [40]:
#print(f'before: {fp16_model.w1.weight}\n')
#optimizer.step()
#print(f'after: {fp16_model.w1.weight}\n')

In [41]:
# copy gradient to FP32 model
fp32_model.w1.weight.grad = fp16_model.w1.weight.grad.float()
fp32_model.w2.weight.grad = fp16_model.w2.weight.grad.float()

In [42]:
print(f'before: {fp32_model.w1.weight}\n')
optimizer.step()
print(f'after: {fp32_model.w1.weight}\n')

before: Parameter containing:
tensor([[ 0.0108,  0.0091,  0.0023,  ..., -0.0054,  0.0047, -0.0132],
        [-0.0140, -0.0182,  0.0112,  ...,  0.0318,  0.0181, -0.0185],
        [-0.0333,  0.0409,  0.0015,  ..., -0.0210, -0.0003, -0.0340],
        ...,
        [ 0.0270,  0.0102,  0.0430,  ..., -0.0134, -0.0132, -0.0054],
        [-0.0426, -0.0003, -0.0204,  ...,  0.0396, -0.0296,  0.0336],
        [ 0.0124, -0.0283,  0.0418,  ..., -0.0426, -0.0158,  0.0190]],
       device='cuda:0', requires_grad=True)

after: Parameter containing:
tensor([[-1.3729,  1.3303, -1.7740,  ..., -0.2675,  0.0519, -0.2462],
        [-0.4777,  0.4250, -0.5844,  ..., -0.0561,  0.0340, -0.0966],
        [ 0.6248, -0.5875,  0.8459,  ...,  0.1036, -0.0227,  0.0767],
        ...,
        [ 1.2351, -1.1442,  1.5943,  ...,  0.2154, -0.0543,  0.1980],
        [-0.0147, -0.0270,  0.0153,  ...,  0.0449, -0.0306,  0.0383],
        [ 0.6768, -0.6630,  0.8949,  ...,  0.0833, -0.0385,  0.1309]],
       device='cuda:0', requ

In [43]:
%%time
import torch
import torch.nn as nn
from torch.cuda import max_memory_allocated

# example input sizes
batch_size, hidden_size = 4, 10000

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.w1 = nn.Linear(hidden_size, hidden_size, bias=False)
        self.w2 = nn.Linear(hidden_size, 1, bias=False)
    
    def forward(self, x):
        z1 = self.w1(x)
        z2 = self.w2(z1)
        return z2

from torch.optim import SGD, Adam

torch.cuda.init()

fp32_model= Net().to("cuda")
optimizer = SGD(fp32_model.parameters(), lr=1e-2)
#optimizer = SGD(fp32_model.parameters(), lr=1e-2, momentum=0.9)
#optimizer = Adam(fp32_model.parameters(), lr=1e-2)
print(f"GPU = {torch.cuda.max_memory_allocated(0) / 1e6} MiB")


# create dummy data (bsz=4, hid=256)
x = torch.randn(batch_size,hidden_size, device="cuda") 
y = torch.tensor([[1.0], [1.0], [1.0], [1.0]], device="cuda")

#torch.cuda.init()
for _ in range(2):
    optimizer.zero_grad()
    z2 = fp32_model(x)
    loss = torch.nn.functional.mse_loss(z2, y)       
    loss.backward()
    optimizer.step()
    memory = max_memory_allocated()
    print(f'step memory allocate: {memory / 1e6:.3f}M')
    #torch.cuda.reset_max_memory_allocated()

GPU = 425.021952 MiB
step memory allocate: 823.834M
step memory allocate: 823.834M
CPU times: user 439 ms, sys: 24.3 ms, total: 463 ms
Wall time: 462 ms


In [44]:
optimizer.param_groups[0]['params']

[Parameter containing:
 tensor([[ 0.0035,  0.0036, -0.0069,  ..., -0.0051,  0.0005, -0.0023],
         [ 0.0095, -0.0065, -0.0096,  ...,  0.0044, -0.0077,  0.0064],
         [-0.0084,  0.0059,  0.0059,  ...,  0.0005,  0.0005, -0.0094],
         ...,
         [ 0.0104,  0.0063,  0.0008,  ..., -0.0070, -0.0085,  0.0130],
         [ 0.0031,  0.0068, -0.0001,  ..., -0.0007, -0.0047, -0.0048],
         [-0.0065,  0.0017,  0.0044,  ..., -0.0092,  0.0024,  0.0034]],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[-0.1063, -0.0443,  0.2530,  ..., -0.1839,  0.2202,  0.1995]],
        device='cuda:0', requires_grad=True)]

### float32 Model

In [45]:
%%time
import torch
import torch.nn as nn
from torch.cuda import max_memory_allocated

# example input sizes
batch_size, hidden_size = 4, 50000

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.w1 = nn.Linear(hidden_size, hidden_size, bias=False)
        self.w2 = nn.Linear(hidden_size, 1, bias=False)
    
    def forward(self, x):
        z1 = self.w1(x)
        z2 = self.w2(z1)
        return z2

from torch.optim import SGD, Adam

torch.cuda.init()
torch.cuda.empty_cache()

fp32_model= Net().to("cuda")
#optimizer = SGD(fp32_model.parameters(), lr=1e-2)
#optimizer = SGD(fp32_model.parameters(), lr=2e-4, momentum=0.9)
optimizer = SGD(fp32_model.parameters(), lr=5e-5, momentum=0.9)
#optimizer = Adam(fp32_model.parameters(), lr=1e-2)
print(f"GPU = {torch.cuda.max_memory_allocated(0) / 1e6} MiB")


# create dummy data (bsz=4, hid=256)
#x = torch.randn(batch_size, hidden_size, device="cuda") 
x = torch.ones(batch_size, hidden_size, device="cuda") 
y = torch.tensor([[3.0], [3.0], [3.0], [3.0]], device="cuda")

torch.cuda.init()
for _ in range(5):
    #x = torch.ones(batch_size, hidden_size, device="cuda")
    optimizer.zero_grad()
    z2 = fp32_model(x)
    loss = torch.nn.functional.mse_loss(z2, y)
    #loss = z2.sum()
    print(f"loss: {loss}")
    loss.backward()
    print(f'before weight: {fp32_model.w1.weight}\n')
    #print(f'before grad: {fp32_model.w1.weight.grad}\n')
    #print(torch.max(fp32_model.w1.weight))
    optimizer.step()
    print(f'after weight: {fp32_model.w1.weight}\n')
    #print(torch.max(fp32_model.w1.weight))
    memory = max_memory_allocated()
    print(f'step memory allocate: {memory / 1e6:.3f}M')
    #torch.cuda.reset_max_memory_allocated()

GPU = 10823.873024 MiB
loss: 6.92418098449707
before weight: Parameter containing:
tensor([[-2.6025e-03, -7.1219e-04,  4.2557e-03,  ..., -2.6104e-03,
         -8.3125e-05, -4.4543e-03],
        [-3.6769e-03, -2.2897e-03,  3.6415e-03,  ...,  2.3379e-03,
          2.1910e-03,  1.1797e-03],
        [-1.4701e-03,  2.5678e-03,  3.1793e-03,  ..., -2.3515e-03,
         -4.3107e-03, -4.1770e-03],
        ...,
        [ 1.1568e-03,  6.1297e-04, -2.2651e-03,  ..., -7.7470e-04,
          1.5991e-03,  4.3360e-03],
        [-3.1571e-03,  1.1334e-03, -2.0015e-03,  ..., -3.6614e-03,
          1.4878e-03, -2.1269e-03],
        [ 2.2894e-04,  1.0092e-04,  3.2273e-03,  ..., -7.4672e-04,
          1.9045e-03, -2.1200e-03]], device='cuda:0', requires_grad=True)

after weight: Parameter containing:
tensor([[-2.6021e-03, -7.1179e-04,  4.2561e-03,  ..., -2.6100e-03,
         -8.2722e-05, -4.4539e-03],
        [-3.6769e-03, -2.2896e-03,  3.6415e-03,  ...,  2.3380e-03,
          2.1911e-03,  1.1797e-03],
     

### float16 Model

In [46]:
%%time
import torch
import torch.nn as nn
from torch.cuda import max_memory_allocated

# example input sizes
batch_size, hidden_size = 4, 50000

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.w1 = nn.Linear(hidden_size, hidden_size, bias=False)
        self.w2 = nn.Linear(hidden_size, 1, bias=False)
    
    def forward(self, x):
        z1 = self.w1(x)
        z2 = self.w2(z1)
        return z2

from torch.optim import SGD, Adam

torch.cuda.init()
torch.cuda.empty_cache()

fp16_model= Net().half().to("cuda")
#optimizer = SGD(fp16_model.parameters(), lr=1e-2)
optimizer = SGD(fp16_model.parameters(), lr=1e-5, momentum=0.9)
#optimizer = Adam(fp16_model.parameters(), lr=1e-2)
print(f"GPU = {torch.cuda.max_memory_allocated(0) / 1e6} MiB")


# create dummy data (bsz=4, hid=256)
#x = torch.randn(batch_size, hidden_size, device="cuda") 
x = torch.ones(batch_size, hidden_size,dtype=torch.half, device="cuda") 
y = torch.tensor([[3.0], [3.0], [3.0], [3.0]], dtype=torch.half, device="cuda")

torch.cuda.init()
for _ in range(5):
    #x = torch.ones(batch_size, hidden_size, device="cuda")
    optimizer.zero_grad()
    z2 = fp16_model(x)
    loss = torch.nn.functional.mse_loss(z2, y)
    #loss = z2.sum()
    print(f"loss: {loss}")
    loss.backward()
    print(f'before weight: {fp32_model.w1.weight}\n')
    #print(f'before grad: {fp32_model.w1.weight.grad}\n')
    #print(torch.max(fp32_model.w1.weight))
    optimizer.step()
    print(f'after weight: {fp32_model.w1.weight}\n')
    #print(torch.max(fp32_model.w1.weight))
    memory = max_memory_allocated()
    print(f'step memory allocate: {memory / 1e6:.3f}M')
    #torch.cuda.reset_max_memory_allocated()

GPU = 35825.01376 MiB
loss: 7.703125
before weight: Parameter containing:
tensor([[-2.6031e-03, -7.1281e-04,  4.2550e-03,  ..., -2.6110e-03,
         -8.3746e-05, -4.4550e-03],
        [-3.6765e-03, -2.2892e-03,  3.6419e-03,  ...,  2.3384e-03,
          2.1914e-03,  1.1801e-03],
        [-1.4705e-03,  2.5674e-03,  3.1789e-03,  ..., -2.3519e-03,
         -4.3111e-03, -4.1774e-03],
        ...,
        [ 1.1563e-03,  6.1254e-04, -2.2656e-03,  ..., -7.7513e-04,
          1.5986e-03,  4.3355e-03],
        [-3.1567e-03,  1.1337e-03, -2.0011e-03,  ..., -3.6610e-03,
          1.4881e-03, -2.1266e-03],
        [ 2.2893e-04,  1.0091e-04,  3.2273e-03,  ..., -7.4673e-04,
          1.9045e-03, -2.1200e-03]], device='cuda:0', requires_grad=True)

after weight: Parameter containing:
tensor([[-2.6031e-03, -7.1281e-04,  4.2550e-03,  ..., -2.6110e-03,
         -8.3746e-05, -4.4550e-03],
        [-3.6765e-03, -2.2892e-03,  3.6419e-03,  ...,  2.3384e-03,
          2.1914e-03,  1.1801e-03],
        [-1.47

### Bfloat16 Model

In [47]:
%%time
import torch
import torch.nn as nn
from torch.cuda import max_memory_allocated

# example input sizes
batch_size, hidden_size = 4, 50000

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.w1 = nn.Linear(hidden_size, hidden_size, bias=False, dtype=torch.bfloat16)
        self.w2 = nn.Linear(hidden_size, 1, bias=False, dtype=torch.bfloat16)
    
    def forward(self, x):
        z1 = self.w1(x)
        z2 = self.w2(z1)
        return z2

from torch.optim import SGD, Adam

torch.cuda.init()
torch.cuda.empty_cache()

bp16_model= Net().to("cuda")
#optimizer = SGD(bp16_model.parameters(), lr=1e-2)
optimizer = SGD(bp16_model.parameters(), lr=1e-5, momentum=0.9)
#optimizer = Adam(bp16_model.parameters(), lr=1e-2)
print(f"GPU = {torch.cuda.max_memory_allocated(0) / 1e6} MiB")


# create dummy data (bsz=4, hid=256)
#x = torch.randn(batch_size, hidden_size, device="cuda") 
x = torch.ones(batch_size, hidden_size,dtype=torch.bfloat16, device="cuda") 
y = torch.tensor([[3.0], [3.0], [3.0], [3.0]], dtype=torch.bfloat16, device="cuda")

torch.cuda.init()
for _ in range(20):
    #x = torch.ones(batch_size, hidden_size, device="cuda")
    optimizer.zero_grad()
    z2 = bp16_model(x)
    loss = torch.nn.functional.mse_loss(z2, y)
    #loss = z2.sum()
    print(f"loss: {loss}")
    loss.backward()
    #print(f'before weight: {fp32_model.w1.weight}\n')
    #print(f'before grad: {fp32_model.w1.weight.grad}\n')
    #print(torch.max(fp32_model.w1.weight))
    optimizer.step()
    #print(f'after weight: {fp32_model.w1.weight}\n')
    #print(torch.max(fp32_model.w1.weight))
    memory = max_memory_allocated()
    print(f'step memory allocate: {memory / 1e6:.3f}M')
    #torch.cuda.reset_max_memory_allocated()

GPU = 40824.714752 MiB
loss: 10.5625
step memory allocate: 45824.815M
loss: 0.15234375
step memory allocate: 45825.216M
loss: 10.375
step memory allocate: 45825.216M
loss: 4.65625
step memory allocate: 45825.216M
loss: 1.4296875
step memory allocate: 45825.216M
loss: 8.25
step memory allocate: 45825.216M
loss: 1.390625
step memory allocate: 45825.216M
loss: 2.75
step memory allocate: 45825.216M
loss: 5.625
step memory allocate: 45825.216M
loss: 0.140625
step memory allocate: 45825.216M
loss: 3.453125
step memory allocate: 45825.216M
loss: 3.421875
step memory allocate: 45825.216M
loss: 0.0791015625
step memory allocate: 45825.216M
loss: 3.515625
step memory allocate: 45825.216M
loss: 1.5625
step memory allocate: 45825.216M
loss: 0.5625
step memory allocate: 45825.216M
loss: 2.875
step memory allocate: 45825.216M
loss: 0.4296875
step memory allocate: 45825.216M
loss: 1.0
step memory allocate: 45825.216M
loss: 1.890625
step memory allocate: 45825.216M
CPU times: user 14 s, sys: 543 ms, t

### Mixed Precision: Autocast 

In [48]:
%%time
import torch
import torch.nn as nn
from torch.cuda import max_memory_allocated

from torch.cuda.amp import autocast 

# example input sizes
batch_size, hidden_size = 4, 50000

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.w1 = nn.Linear(hidden_size, hidden_size, bias=False)
        self.w2 = nn.Linear(hidden_size, 1, bias=False)
    
    def forward(self, x):
        z1 = self.w1(x)
        z2 = self.w2(z1)
        return z2

from torch.optim import SGD, Adam

torch.cuda.init()
torch.cuda.empty_cache()

fp32_model= Net().to("cuda")
#optimizer = SGD(fp32_model.parameters(), lr=1e-2)
#optimizer = SGD(fp32_model.parameters(), lr=1e-4, momentum=0.9) # diverge
optimizer = SGD(fp32_model.parameters(), lr=1e-5, momentum=0.9)
#optimizer = Adam(fp32_model.parameters(), lr=1e-2)
print(f"GPU = {torch.cuda.max_memory_allocated(0) / 1e6} MiB")

# create dummy data (bsz=4, hid=256)
#x = torch.randn(batch_size, hidden_size, device="cuda") 
x = torch.ones(batch_size, hidden_size, device="cuda") 
y = torch.tensor([[3.0], [3.0], [3.0], [3.0]], device="cuda")

torch.cuda.init()
for _ in range(20):
  with autocast():
  #with autocast(cache_enabled=False):
    #x = torch.ones(batch_size, hidden_size, device="cuda")
    optimizer.zero_grad()
    z2 = fp32_model(x)
    loss = torch.nn.functional.mse_loss(z2, y)
    #loss = z2.sum()
    print(f"loss: {loss}")
    loss.backward()
    #print(f'before weight: {fp32_model.w1.weight}\n')
    #print(f'before grad: {fp32_model.w1.weight.grad}\n')
    #print(torch.max(fp32_model.w1.weight))
    optimizer.step()
    #print(f'after weight: {fp32_model.w1.weight}\n')
    #print(torch.max(fp32_model.w1.weight))
    memory = max_memory_allocated()
    print(f'step memory allocate: {memory / 1e6:.3f}M')
    #torch.cuda.reset_max_memory_allocated()

GPU = 55825.015296 MiB
loss: 9.86275863647461
step memory allocate: 55825.415M
loss: 1.0837135314941406
step memory allocate: 60825.416M
loss: 2.3807525634765625
step memory allocate: 60825.416M
loss: 8.064712524414062
step memory allocate: 60825.416M
loss: 4.41656494140625
step memory allocate: 60825.416M
loss: 0.00152587890625
step memory allocate: 60825.416M
loss: 3.3886194229125977
step memory allocate: 60825.416M
loss: 5.322843551635742
step memory allocate: 60825.416M
loss: 1.42877197265625
step memory allocate: 60825.416M
loss: 0.3571929931640625
step memory allocate: 60825.416M
loss: 3.28515625
step memory allocate: 60825.416M
loss: 2.9139556884765625
step memory allocate: 60825.416M
loss: 0.2308502197265625
step memory allocate: 60825.416M
loss: 0.8825721740722656
step memory allocate: 60825.416M
loss: 2.5431528091430664
step memory allocate: 60825.416M
loss: 1.2876930236816406
step memory allocate: 60825.416M
loss: 0.00054931640625
step memory allocate: 60825.416M
loss: 1.104

### Autocast + torch.cuda.amp.GradScaler()

In [49]:
%%time
import torch
import torch.nn as nn
from torch.cuda import max_memory_allocated

from torch.cuda.amp import autocast 

# example input sizes
batch_size, hidden_size = 4, 50000

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.w1 = nn.Linear(hidden_size, hidden_size, bias=False)
        self.w2 = nn.Linear(hidden_size, 1, bias=False)
    
    def forward(self, x):
        z1 = self.w1(x)
        z2 = self.w2(z1)
        return z2

from torch.optim import SGD, Adam

torch.cuda.init()
torch.cuda.empty_cache()

fp32_model= Net().to("cuda")
#optimizer = SGD(fp32_model.parameters(), lr=1e-2)
#optimizer = SGD(fp32_model.parameters(), lr=1e-4, momentum=0.9) #diverge
optimizer = SGD(fp32_model.parameters(), lr=1e-5, momentum=0.9)
#optimizer = Adam(fp32_model.parameters(), lr=1e-2)
print(f"GPU = {torch.cuda.max_memory_allocated(0) / 1e6} MiB")

# create dummy data (bsz=4, hid=256)
#x = torch.randn(batch_size, hidden_size, device="cuda") 
x = torch.ones(batch_size, hidden_size, device="cuda") 
y = torch.tensor([[3.0], [3.0], [3.0], [3.0]], device="cuda")

torch.cuda.init()

scaler = torch.cuda.amp.GradScaler()

for _ in range(20):
    optimizer.zero_grad()
    with autocast():
        z2 = fp32_model(x)
        loss = torch.nn.functional.mse_loss(z2, y)
        
    print(f"loss: {loss}")
    scaler.scale(loss).backward()
    #print(f'before weight: {fp32_model.w1.weight}\n')
    #print(f'before grad: {fp32_model.w1.weight.grad}\n')
    #print(torch.max(fp32_model.w1.weight))
    scaler.step(optimizer)
    scaler.update()
    #print(f'after weight: {fp32_model.w1.weight}\n')
    #print(torch.max(fp32_model.w1.weight))
    memory = max_memory_allocated()
    print(f'step memory allocate: {memory / 1e6:.3f}M')
    #torch.cuda.reset_max_memory_allocated()

GPU = 60825.515008 MiB
loss: 14.198324203491211
step memory allocate: 60825.515M
loss: 14.198324203491211
step memory allocate: 60825.515M
loss: 14.198324203491211
step memory allocate: 60825.515M
loss: 14.198324203491211
step memory allocate: 60825.515M
loss: 14.198324203491211
step memory allocate: 60825.515M
loss: 14.198324203491211
step memory allocate: 60825.515M
loss: 1.5478858947753906
step memory allocate: 60825.515M
loss: 3.457275390625
step memory allocate: 60825.515M
loss: 11.6025390625
step memory allocate: 60825.515M
loss: 6.328369140625
step memory allocate: 60825.515M
loss: 0.001102447509765625
step memory allocate: 60825.515M
loss: 4.914188385009766
step memory allocate: 60825.515M
loss: 7.647331237792969
step memory allocate: 60825.515M
loss: 2.0272865295410156
step memory allocate: 60825.515M
loss: 0.5250587463378906
step memory allocate: 60825.515M
loss: 4.7340240478515625
step memory allocate: 60825.515M
loss: 4.1737213134765625
step memory allocate: 60825.515M
loss

### NVIDIA AMP (Apex Mixed Precision)

In [50]:
%%time
import torch
import torch.nn as nn
from torch.cuda import max_memory_allocated

#from torch.cuda.amp import autocast 

from apex import amp

# example input sizes
batch_size, hidden_size = 4, 50000

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.w1 = nn.Linear(hidden_size, hidden_size, bias=False)
        self.w2 = nn.Linear(hidden_size, 1, bias=False)
    
    def forward(self, x):
        z1 = self.w1(x)
        z2 = self.w2(z1)
        return z2

from torch.optim import SGD, Adam

torch.cuda.init()
torch.cuda.empty_cache()

fp32_model= Net().to("cuda")
#optimizer = SGD(fp32_model.parameters(), lr=1e-2)
#optimizer = SGD(fp32_model.parameters(), lr=1e-4, momentum=0.9) #diverge
optimizer = SGD(fp32_model.parameters(), lr=1e-5, momentum=0.9)
#optimizer = Adam(fp32_model.parameters(), lr=1e-2)
print(f"GPU = {torch.cuda.max_memory_allocated(0) / 1e6} MiB")

# Allow Amp to perform casts as required by the opt_level
fp32_model, optimizer = amp.initialize(fp32_model, optimizer, opt_level="O3")

# create dummy data (bsz=4, hid=256)
#x = torch.randn(batch_size, hidden_size, device="cuda") 
x = torch.ones(batch_size, hidden_size, device="cuda") 
y = torch.tensor([[3.0], [3.0], [3.0], [3.0]], device="cuda")

torch.cuda.init()

#scaler = torch.cuda.amp.GradScaler()

for _ in range(5):
    optimizer.zero_grad()
    z2 = fp32_model(x)
    loss = torch.nn.functional.mse_loss(z2, y)
        
    print(f"loss: {loss}")
    #scaler.scale(loss).backward()
    # loss.backward() becomes:
    with amp.scale_loss(loss, optimizer) as scaled_loss:
        scaled_loss.backward()
    print(f'before weight: {fp32_model.w1.weight}\n')
    #print(f'before grad: {fp32_model.w1.weight.grad}\n')
    #print(torch.max(fp32_model.w1.weight))
    #scaler.step(optimizer)
    #scaler.update()
    optimizer.step()
    print(f'after weight: {fp32_model.w1.weight}\n')
    #print(torch.max(fp32_model.w1.weight))
    memory = max_memory_allocated()
    print(f'step memory allocate: {memory / 1e6:.3f}M')
    #torch.cuda.reset_max_memory_allocated()

GPU = 60825.516032 MiB
Selected optimization level O3:  Pure FP16 training.
Defaults for this optimization level are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0
loss: 8.989714622497559
before weight: Parameter containing:
tensor([[-9.0981e-04,  3.0441e-03, -3.2597e-03,  ...,  1.1253e-03,
         -2.5101e-03, -1.2150e-03],
        [-1.5602e-03, -6.0678e-05,  3.2139e-03,  ..., -1.6451e-03,
         -3.7518e-03,  2.5578e-03],
        [-1.6747e-03, -4.8459e-05, -2.9564e-03,  ..., -3

/scratch/qualis/miniconda3/envs/dp/lib/python3.10/site-packages/apex/__init__.py:68: DeprecatedFeatureWarning: apex.amp is deprecated and will be removed by the end of February 2023. Use [PyTorch AMP](https://pytorch.org/docs/stable/amp.html)
  warnings.warn(msg, DeprecatedFeatureWarning)
/scratch/qualis/miniconda3/envs/dp/lib/python3.10/site-packages/apex/amp/_process_optimizer.py:344: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at /opt/conda/conda-bld/pytorch_1699449201336/work/torch/csrc/tensor/python_tensor.cpp:83.)
  optimizer._amp_stash.dummy_overflow_buf = torch.cuda.IntTensor([0]);


before weight: Parameter containing:
tensor([[-9.0981e-04,  3.0441e-03, -3.2597e-03,  ...,  1.1253e-03,
         -2.5101e-03, -1.2150e-03],
        [-1.5602e-03, -6.0260e-05,  3.2139e-03,  ..., -1.6451e-03,
         -3.7518e-03,  2.5578e-03],
        [-1.6747e-03, -4.8459e-05, -2.9564e-03,  ..., -3.3092e-03,
          4.4136e-03,  2.4395e-03],
        ...,
        [ 2.4204e-03,  1.4067e-03,  2.3711e-04,  ..., -3.6182e-03,
         -1.0786e-03, -5.3358e-04],
        [-4.2114e-03,  2.1019e-03,  4.3182e-03,  ..., -2.2354e-03,
          4.4403e-03,  1.0519e-03],
        [-2.9945e-04, -2.4815e-03, -4.1542e-03,  ..., -1.5650e-03,
          4.3640e-03,  2.1992e-03]], device='cuda:0', dtype=torch.float16,
       requires_grad=True)

after weight: Parameter containing:
tensor([[-9.0981e-04,  3.0441e-03, -3.2597e-03,  ...,  1.1253e-03,
         -2.5101e-03, -1.2150e-03],
        [-1.5602e-03, -6.0201e-05,  3.2139e-03,  ..., -1.6451e-03,
         -3.7518e-03,  2.5578e-03],
        [-1.6747e-03, -

In [51]:
%%time
import torch
import torch.nn as nn
from torch.cuda import max_memory_allocated

from torch.cuda.amp import autocast 

# example input sizes
batch_size, hidden_size = 4, 50000

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.w1 = nn.Linear(hidden_size, hidden_size, bias=False)
        self.w2 = nn.Linear(hidden_size, 1, bias=False)
    
    def forward(self, x):
        z1 = self.w1(x)
        z2 = self.w2(z1)
        return z2

from torch.optim import SGD, Adam

torch.cuda.init()
torch.cuda.empty_cache()

fp32_model= Net().to("cuda")
print(f"GPU = {torch.cuda.max_memory_allocated(0) / 1e6} MiB")

#optimizer = SGD(fp32_model.parameters(), lr=1e-2)
#optimizer = SGD(fp32_model.parameters(), lr=1e-4, momentum=0.9) #diverge
optimizer = SGD(fp32_model.parameters(), lr=1e-5, momentum=0.9)
#optimizer = Adam(fp32_model.parameters(), lr=1e-2)

fp16_model = Net().half().to("cuda")
fp16_model.load_state_dict(fp32_model.state_dict())
print(f"GPU = {torch.cuda.max_memory_allocated(0) / 1e6} MiB")

# create dummy data (bsz=4, hid=256)
#x = torch.randn(batch_size, hidden_size, device="cuda") 
x = torch.ones(batch_size, hidden_size, dtype=torch.half, device="cuda") 
y = torch.tensor([[3.0], [3.0], [3.0], [3.0]], dtype=torch.half, device="cuda")

torch.cuda.init()

for _ in range(5):
    optimizer.zero_grad()
    z2 = fp16_model(x)
    loss = torch.nn.functional.mse_loss(z2, y)    
    #loss *= 1024
    print(f"loss: {loss}")
    loss.backward()
    # copy gradient to FP32 model
    fp32_model.w1.weight.grad = fp16_model.w1.weight.grad.float()
    fp32_model.w2.weight.grad = fp16_model.w2.weight.grad.float()
    print(f'before weight: {fp32_model.w1.weight}\n')
    #print(f'before grad: {fp32_model.w1.weight.grad}\n')
    #print(torch.max(fp32_model.w1.weight))
    optimizer.step()
    print(f'after weight: {fp32_model.w1.weight}\n')
    #print(torch.max(fp32_model.w1.weight))
    memory = max_memory_allocated()
    print(f'step memory allocate: {memory / 1e6:.3f}M')
    #torch.cuda.reset_max_memory_allocated()
    fp16_model.load_state_dict(fp32_model.state_dict())

GPU = 60825.516032 MiB
GPU = 60825.516032 MiB
loss: 10.28125
before weight: Parameter containing:
tensor([[ 0.0028,  0.0036, -0.0021,  ...,  0.0016,  0.0030, -0.0012],
        [ 0.0024,  0.0002, -0.0032,  ...,  0.0003, -0.0013, -0.0005],
        [ 0.0004, -0.0026,  0.0036,  ..., -0.0022,  0.0041, -0.0003],
        ...,
        [ 0.0012,  0.0032,  0.0012,  ..., -0.0024,  0.0034,  0.0028],
        [ 0.0031, -0.0026, -0.0041,  ..., -0.0026, -0.0035, -0.0039],
        [ 0.0035, -0.0010, -0.0007,  ...,  0.0025, -0.0026,  0.0012]],
       device='cuda:0', requires_grad=True)

after weight: Parameter containing:
tensor([[ 0.0028,  0.0036, -0.0021,  ...,  0.0016,  0.0030, -0.0012],
        [ 0.0024,  0.0002, -0.0032,  ...,  0.0003, -0.0013, -0.0005],
        [ 0.0004, -0.0026,  0.0036,  ..., -0.0022,  0.0041, -0.0003],
        ...,
        [ 0.0012,  0.0032,  0.0012,  ..., -0.0024,  0.0034,  0.0028],
        [ 0.0031, -0.0026, -0.0041,  ..., -0.0026, -0.0035, -0.0039],
        [ 0.0035, -0.001

In [53]:
!pwd

/scratch/qualis/large-scale-lm-tutorials/src


In [54]:
# Zero-3 Video 
from IPython.display import HTML

HTML("""
<div align="middle">
<video width="80%" controls>
      <source src="../images/zero_video.mp4" type="video/mp4">
</video></div>""")